In [166]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


### EDA

In [167]:
original_data = pd.read_csv('insurance.csv')
df = original_data.copy()

In [168]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [169]:
df.duplicated().sum()

1

In [170]:
df.drop_duplicates(inplace=True)

In [171]:
df.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

### DATA PREPROCESSING

In [172]:
def ident_cat_num(df): 
    # ESTA FUNCION IDENTIFICA SI LOS DATOS SON DE TIPO NUMERICO O CATEGORICO
    import pandas as pd
    #CREAMOS UNA LISTA PARA DESPUÉS ALMACENAR LOS NOMBRES DE LAS COLUMNAS QUE SEAN NUMERICAS
    list_numericas = [] 
    #CREAMOS UNA LISTA PARA DESPUÉS ALMACENAR LOS NOMBRES DE LAS COLUMNAS QUE SEAN CATEGORICAS
    list_categoricas = []
    #RECORREMOS LAS COLUMNAS DE LA BASE DE DATOS
    for i in df.columns: 
        #IDENTIFICAMOS SI LOS DATOS QUE CONTIENE LA COLUMNA SON NUMERICOS
        if pd.api.types.is_numeric_dtype(df[i])== True:
            #ALMACENAMOS LOS NOMBRES DE LAS COLUMNAS QUE SEAN NUM
            list_numericas.append(i)
        else:
            #SINO LOS ALMACENAMOS LOS NOMBRES DE LAS COLUMNAS QUE SEAN CATEGORICAS
            list_categoricas.append(i)
    return list_numericas,list_categoricas

In [173]:
ident_cat_num(df)


(['age', 'bmi', 'children', 'charges'], ['sex', 'smoker', 'region'])

In [174]:
list_numericas = ident_cat_num(df)[0]
list_categoricas = ident_cat_num(df)[1]


In [175]:
onehot=OneHotEncoder()
a=onehot.fit_transform(df[list_categoricas])
encoded_df = pd.DataFrame(a.toarray(), columns=onehot.get_feature_names_out(['sex','smoker','region']))
df.reset_index(drop=True, inplace=True)
encoded_df.reset_index(drop=True, inplace=True)
df=pd.concat([df,encoded_df],axis=1)
df.drop(columns=list_categoricas,inplace=True)
print(df.shape)
df.sample(5)



(1337, 12)


,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
790,19,27.600,0,1252.40700,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
287,63,26.220,0,14256.19280,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
693,27,34.800,1,3577.99900,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1266,24,31.065,0,34254.05335,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
902,49,36.850,0,8125.78450,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0


In [176]:
X = df.drop(columns=['charges'],inplace= False)
y= df['charges']
print(X.shape)
print(y.shape)

(1337, 11)
(1337,)


In [177]:
kfold = KFold(n_splits=15)
    

### MODEL EVALUATION

In [185]:
modelo = RandomForestRegressor(n_estimators=101)
media_errores=[]


for i_train, i_test in kfold.split(df):
    X_train,X_test,y_train,y_test = X.iloc[i_train,:],X.iloc[i_test,:],y.iloc[i_train],y.iloc[i_test]
    estandarizador = StandardScaler()
    estandarizador.fit(X_train)
    X_train_std=estandarizador.transform(X_train)
    X_test_std=estandarizador.transform(X_test)
    modelo.fit(X_train_std,y_train)
    predictions = modelo.predict(X_test_std)
    media_errores.append(mean_squared_error(y_test,predictions))


In [179]:
MSE = sum(media_errores) / len(media_errores)
MSE

23643657.349432275

### FINAL MODEL

In [183]:
estandarizador.fit(X)
X_std=estandarizador.transform(X)
modelo.fit(X_std,y)
predictions= modelo.predict(X_std)
df['Predictions'] = predictions


In [186]:
df[['charges','Predictions']].sample(50)

,charges,Predictions
396,8522.00300,9078.773913
737,36189.10170,35715.525023
472,1744.46500,1996.142337
180,11735.87905,11825.555258
1225,6640.54485,7411.531562
0,16884.92400,16968.956906
62,30166.61817,24397.534457
379,27000.98473,22481.435741
355,24603.04837,19066.441055
626,11326.71487,8987.290235
